# Author imformation:
Fan Shengzhe, Shanghaijiaotong University, Shanghai, China  
Email: fanshengzhe@sjtu.edu.cn

# 1.Open Files

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import os
os.chdir("drive/My Drive/reco/agri-machine-reco")

In [ ]:
import pandas as pd
log_table = pd.read_csv('./data/log_table.csv')
log_table

,event_time,event_type,item_id,power,category_id,region,price,user_id,user_session
0,2018-01-01 00:00:00,intent,0,0.073838,0,0.0,0.081670,0,26dd6e6e-4dac-4778-8d2c-92e149dab885
1,2018-01-01 00:16:17,intent,1,0.087144,0,0.0,0.081670,0,26dd6e6e-4dac-4778-8d2c-92e149dab885
2,2018-01-01 00:38:00,intent,2,0.070977,0,0.0,0.081670,0,26dd6e6e-4dac-4778-8d2c-92e149dab885
3,2018-01-01 13:45:25,intent,3,0.053446,0,0.0,0.081670,0,26dd6e6e-4dac-4778-8d2c-92e149dab885
4,2018-01-01 14:28:51,intent,4,0.064256,0,0.0,0.081670,0,26dd6e6e-4dac-4778-8d2c-92e149dab885
...,...,...,...,...,...,...,...,...,...
422121,2020-12-31 21:00:47,view,10356,0.068162,0,15.0,0.085841,54532,91404fcc-31fa-4134-8dd0-2f85ed6c8964
422122,2020-12-31 21:06:13,view,25183,0.106500,0,15.0,0.090755,54532,60b32523-a303-43f7-8cd7-b69d0fd18405
422123,2020-12-31 21:17:05,view,2889,0.118093,0,15.0,0.102280,54532,e66ceee2-dc38-418c-a05f-52cf07ad2e43
422124,2020-12-31 21:17:05,view,2886,0.083645,0,15.0,0.085841,54532,75abd9a1-e3ab-47c8-8c0d-09d8d21d1b05


In [ ]:
log_table.describe()

,item_id,power,category_id,region,price,user_id
count,422126.000000,422126.000000,422126.000000,255106.000000,422126.000000,422126.000000
mean,9109.396718,0.099193,3.397324,29.426866,0.099149,23667.572097
std,8460.390361,0.055905,9.173678,35.446887,0.055026,16886.295280
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2155.000000,0.078332,0.000000,6.000000,0.080897,7895.000000
50%,6271.000000,0.088290,0.000000,17.000000,0.086274,21765.000000
75%,14089.000000,0.100626,0.000000,38.000000,0.095884,38250.000000
max,32783.000000,1.000000,54.000000,221.000000,1.000000,54532.000000


# 2.Get timestamp

In [ ]:
import time
def get_timestamp(time_str):
  timeArray = time.strptime(time_str, "%Y-%m-%d %H:%M:%S")
  timeStamp = int(time.mktime(timeArray))
  return timeStamp

def get_timestr(timestamp):
  timeArray = time.localtime(timestamp)
  return time.strftime("%Y-%m-%d %H:%M:%S", timeArray)

In [ ]:
log_table['event_time'] = log_table['event_time'].map(get_timestamp)

# 3.Encode category features

In [ ]:
print(len(log_table['user_id'].unique()))
print(len(log_table['user_session'].unique()))
print(log_table['user_session'].isna().sum())
print()
print(len(log_table['item_id'].unique()))
print(len(log_table['category_id'].unique()))


54533
86248
40

32784
55


In [ ]:
log_table['item_id'].describe()

count    422126.000000
mean       9109.396718
std        8460.390361
min           0.000000
25%        2155.000000
50%        6271.000000
75%       14089.000000
max       32783.000000
Name: item_id, dtype: float64

## 3.1 session

In [ ]:
us_dict = {}
for index, user_session in enumerate(log_table['user_session'].unique()):
  us_dict[user_session] = index

def us_encode(user_session):
  return us_dict[user_session]

log_table['user_session'] = log_table['user_session'].map(us_encode)

## 3.2 event_type

In [ ]:
event_type_map = {'remove_intent': 0, 'view': 1, 'intent': 2, 'deal': 3}
rev_event_type_map = {0:'remove_intent', 1: 'view', 2: 'intent', 3: 'deal'}

def event_encode(event_type):
  return event_type_map[event_type]

def event_decode(event_type):
  return rev_event_type_map[event_type]
log_table['event_type'] = log_table['event_type'].map(event_encode)

## 3.3 user id

In [ ]:
uid_dict = {}
for index, uid in enumerate(log_table['user_id'].unique()):
  uid_dict[uid] = index

def uid_encode(uid):
  return uid_dict[uid]

log_table['user_id'] = log_table['user_id'].map(uid_encode)

## 3.4 item id

In [ ]:
iid_dict = {}
for index, iid in enumerate(log_table['item_id'].unique()):
  iid_dict[iid] = index

def iid_encode(iid):
  return iid_dict[iid]

log_table['item_id'] = log_table['item_id'].map(iid_encode)

## 3.5 category id

In [ ]:
cid_dict = {}
for index, cid in enumerate(log_table['category_id'].unique()):
  cid_dict[cid] = index

def cid_encode(cid):
  return cid_dict[cid]

log_table['category_id'] = log_table['category_id'].map(cid_encode)

# 4.Process nan value

In [ ]:
log_table['category_id'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54])

In [ ]:
log_table['region'].unique()

array([  0.,   1.,  nan,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,
        10.,  11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,
        21.,  22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,
        32.,  33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,
        43.,  44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,
        54.,  55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,
        65.,  66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,
        76.,  77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,
        87.,  88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,
        98.,  99., 100., 101., 102., 103., 104., 105., 106., 107., 108.,
       109., 110., 111., 112., 113., 114., 115., 116., 117., 118., 119.,
       120., 121., 122., 123., 124., 125., 126., 127., 128., 129., 130.,
       131., 132., 133., 134., 135., 136., 137., 138., 139., 140., 141.,
       142., 143., 144., 145., 146., 147., 148., 14

In [ ]:
log_table['is_nan_region'] = log_table['region'].isna().map(int)
log_table

,event_time,event_type,item_id,power,category_id,region,price,user_id,user_session,is_nan_region
0,1514764800,2,0,0.073838,0,0.0,0.081670,0,0,0
1,1514765777,2,1,0.087144,0,0.0,0.081670,0,0,0
2,1514767080,2,2,0.070977,0,0.0,0.081670,0,0,0
3,1514814325,2,3,0.053446,0,0.0,0.081670,0,0,0
4,1514816931,2,4,0.064256,0,0.0,0.081670,0,0,0
...,...,...,...,...,...,...,...,...,...,...
422121,1609448447,1,10356,0.068162,0,15.0,0.085841,54532,86243,0
422122,1609448773,1,25183,0.106500,0,15.0,0.090755,54532,86244,0
422123,1609449425,1,2889,0.118093,0,15.0,0.102280,54532,86245,0
422124,1609449425,1,2886,0.083645,0,15.0,0.085841,54532,86246,0


In [ ]:
sum(log_table['is_nan_region'])

167020

In [ ]:
sum(log_table['region'].isna())

167020

# 5.Save result

In [ ]:
len(log_table['user_id'].unique())

54533

In [ ]:
log_table['event_time'] = log_table['event_time'].map(get_timestr)
log_table['event_type'] = log_table['event_type'].map(event_decode)

In [ ]:
log_table.head()

,event_time,event_type,item_id,power,category_id,region,price,user_id,user_session,is_nan_region
0,2018-01-01 00:00:00,intent,0,0.073838,0,0.0,0.08167,0,0,0
1,2018-01-01 00:16:17,intent,1,0.087144,0,0.0,0.08167,0,0,0
2,2018-01-01 00:38:00,intent,2,0.070977,0,0.0,0.08167,0,0,0
3,2018-01-01 13:45:25,intent,3,0.053446,0,0.0,0.08167,0,0,0
4,2018-01-01 14:28:51,intent,4,0.064256,0,0.0,0.08167,0,0,0


In [ ]:
log_table.describe()

,item_id,power,category_id,region,price,user_id,user_session,is_nan_region
count,422126.000000,422126.000000,422126.000000,255106.000000,422126.000000,422126.000000,422126.000000,422126.000000
mean,9109.396718,0.099193,3.397324,29.426866,0.099149,23667.572097,41495.441534,0.395664
std,8460.390361,0.055905,9.173678,35.446887,0.055026,16886.295280,25314.781381,0.488993
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2155.000000,0.078332,0.000000,6.000000,0.080897,7895.000000,19336.250000,0.000000
50%,6271.000000,0.088290,0.000000,17.000000,0.086274,21765.000000,40376.000000,0.000000
75%,14089.000000,0.100626,0.000000,38.000000,0.095884,38250.000000,63569.000000,1.000000
max,32783.000000,1.000000,54.000000,221.000000,1.000000,54532.000000,86247.000000,1.000000


In [ ]:
if not os.path.exists('./cache'):
  os.mkdir('./cache')

In [ ]:
log_table.to_csv('./cache/log_table.csv', index=False)